In [6]:
import pandas as pd
from pathlib import Path

# ================== CONFIG (edit if needed) ==================
CYGNSS_DIR = Path("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand/2021_06_v2")
WATER_DIR  = Path("/mnt/cephfs-mount/chenchen/water_body_fraction/2021_06")
OUT_DIR    = Path("/mnt/cephfs-mount/chenchen/water_body_fraction_with_landcover&slope/2021_06")
ROUND_DECIMALS = 4          # rounding for lon/lat match
PREVIEW = False             # True = dry run (no writing)
NEEDED_COLS = ["modis_land_cover", "srtm_slope"]
# ============================================================

def pick_join_keys(df_a: pd.DataFrame, df_b: pd.DataFrame):
    """Pick best available join keys shared by both DataFrames."""
    k1 = ["sample", "ddm"]
    if all(k in df_a.columns for k in k1) and all(k in df_b.columns for k in k1):
        return k1, "sample+ddm"

    k2 = ["sp_lon", "sp_lat"]
    if all(k in df_a.columns for k in k2) and all(k in df_b.columns for k in k2):
        return k2, "lon+lat"

    return None, "index"  # fallback

def safe_read_csv(p: Path):
    try:
        return pd.read_csv(p)
    except Exception as e:
        print(f"[ERROR] reading {p}: {e}")
        return None

def ensure_outdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# ---------- main ----------
ensure_outdir(OUT_DIR)

water_files = sorted(WATER_DIR.glob("*.csv"))
if not water_files:
    raise SystemExit(f"[ABORT] No CSVs found in {WATER_DIR}")

summary_rows = []
print(f"Found {len(water_files)} water CSVs. Starting merge...\n")

for wpath in water_files:
    fname = wpath.name
    cpath = CYGNSS_DIR / fname

    wdf = safe_read_csv(wpath)
    if wdf is None:
        summary_rows.append({"file": fname, "status": "water_read_error"})
        continue

    if not cpath.exists():
        print(f"[SKIP] Missing CYGNSS: {fname}")
        summary_rows.append({
            "file": fname, "status": "missing_cygnss",
            "water_rows": len(wdf), "cygnss_rows": None,
            "matched": 0, "unmatched": len(wdf), "join_key": None
        })
        continue

    cdf = safe_read_csv(cpath)
    if cdf is None:
        summary_rows.append({"file": fname, "status": "cygnss_read_error"})
        continue

    # Check CYGNSS has needed columns
    missing = [c for c in NEEDED_COLS if c not in cdf.columns]
    if missing:
        print(f"[SKIP] {fname}: CYGNSS missing columns {missing}")
        summary_rows.append({
            "file": fname, "status": "missing_columns",
            "water_rows": len(wdf), "cygnss_rows": len(cdf),
            "matched": 0, "unmatched": len(wdf), "join_key": None
        })
        continue

    # Decide join keys
    join_keys, key_name = pick_join_keys(wdf, cdf)

    if join_keys is None:
        # Fallback: align by row index
        min_len = min(len(wdf), len(cdf))
        matched = min_len
        unmatched = len(wdf) - min_len
        if not PREVIEW:
            # Ensure columns exist, then fill first min_len rows
            for col in NEEDED_COLS:
                if col not in wdf.columns:
                    wdf[col] = pd.NA
                wdf.loc[:min_len-1, col] = cdf.loc[:min_len-1, col].values
    else:
        wdf_keyed = wdf.copy()
        cdf_keyed = cdf.copy()

        if key_name == "lon+lat":
            for col in ["sp_lon", "sp_lat"]:
                wdf_keyed[col] = wdf_keyed[col].round(ROUND_DECIMALS)
                cdf_keyed[col] = cdf_keyed[col].round(ROUND_DECIMALS)

        # Deduplicate CYGNSS rows on key to avoid m:n joins
        cdf_keep = cdf_keyed[join_keys + NEEDED_COLS].drop_duplicates(subset=join_keys, keep="first")
        merged = wdf_keyed.merge(cdf_keep, on=join_keys, how="left", validate="m:1")
        matched = int(merged[NEEDED_COLS[0]].notna().sum())
        unmatched = len(merged) - matched
        if not PREVIEW:
            wdf = merged

    status = "preview" if PREVIEW else "ok"
    if not PREVIEW:
        out_path = OUT_DIR / fname
        try:
            wdf.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")
            status = "write_error"

    summary_rows.append({
        "file": fname, "status": status,
        "water_rows": len(wdf), "cygnss_rows": len(cdf),
        "matched": int(matched), "unmatched": int(unmatched),
        "join_key": key_name
    })

    print(f"[{status}] {fname} | key={key_name} | matched {matched} / {len(wdf)}")

print("\n✅ Done.")
print("\nStatus counts:")

Found 30 water CSVs. Starting merge...

[ok] 20210401.csv | key=sample+ddm | matched 2658 / 2658
[ok] 20210402.csv | key=sample+ddm | matched 2237 / 2237
[ok] 20210403.csv | key=sample+ddm | matched 2500 / 2500
[ok] 20210404.csv | key=sample+ddm | matched 2394 / 2394
[ok] 20210405.csv | key=sample+ddm | matched 2468 / 2468
[ok] 20210406.csv | key=sample+ddm | matched 2581 / 2581
[ok] 20210407.csv | key=sample+ddm | matched 2367 / 2367
[ok] 20210408.csv | key=sample+ddm | matched 2492 / 2492
[ok] 20210409.csv | key=sample+ddm | matched 2379 / 2379
[ok] 20210410.csv | key=sample+ddm | matched 2326 / 2326
[ok] 20210411.csv | key=sample+ddm | matched 2559 / 2559
[ok] 20210412.csv | key=sample+ddm | matched 2615 / 2615
[ok] 20210413.csv | key=sample+ddm | matched 2711 / 2711
[ok] 20210414.csv | key=sample+ddm | matched 2199 / 2199
[ok] 20210415.csv | key=sample+ddm | matched 2440 / 2440
[ok] 20210416.csv | key=sample+ddm | matched 2227 / 2227
[ok] 20210417.csv | key=sample+ddm | matched 231

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, re, sys
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import xarray as xr

# ========= User settings =========
input_root   = "/mnt/cephfs-mount/chenchen/CygnssData"        # NC root (month folders)
output_root  = ("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full")  # Existing CSV root (month folders)
years_filter = {2018,2019}  # e.g., {2023,2024} or None for all

# Write mode: overwrite or write to sibling folder
SAFE_OVERWRITE   = False
OUTPUT_SUFFIX    = "_v3"       # used if SAFE_OVERWRITE=False (writes YYYY_MM_v2)
USE_GZIP_OUTPUT  = False       # set True to write *.csv.gz

# Rounding for stable float keys (must match producer tolerance)
LON_LAT_ROUND = 6   # <- matches your demo

# ======== New variables to add (replace LC+DEM) ========
NEW_VARS = [
    "gps_ant_gain_db_i",
    "sp_rx_gain",
    "gps_tx_power_db_w",
    "ddm_ant",
    "inst_gain",
    "tx_to_sp_range",
    "rx_to_sp_range",
    "ddm_noise_floor",
    "nbrcs_scatter_area",
]

# ======== Pandas nullable-int detection / dtype helper (kept for structure) ========
def get_nullable_int16_dtype():
    try:
        return pd.Int16Dtype()  # extension dtype (handles NaN)
    except Exception:
        return None

NULLABLE_INT16_DTYPE = get_nullable_int16_dtype()

# ========= Helpers =========
def parse_month_folder_name(name):
    m = re.match(r'^(\d{4})[-_]?(\d{2})$', name)
    if not m: return None, None
    y, mm = int(m.group(1)), int(m.group(2))
    return (y, mm) if 1 <= mm <= 12 else (None, None)

def find_month_dirs(root, years_filter=None):
    """Return (name, year, month, full_path). Accepts YYYYMM / YYYY_MM / YYYY-MM directly under root."""
    out = []
    for name in sorted(os.listdir(root)):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        y, m = parse_month_folder_name(name)
        if y is None:
            continue
        if (years_filter is None) or (y in years_filter):
            out.append((name, y, m, p))
    return out

def extract_date_from_filename(fname: str):
    m = re.search(r's(\d{8})', fname)
    return m.group(1) if m else None

def norm_lon(arr):
    # match your demo (> 180)
    return np.where(arr > 180, arr - 360, arr)

def open_nc(path):
    # Prefer netcdf4 engine if available
    try:
        return xr.open_dataset(path, engine='netcdf4', mask_and_scale=True)
    except Exception:
        return xr.open_dataset(path, mask_and_scale=True)

def collect_nc_files_for_month(month_nc_dir):
    """Find daily NC files (grouped by date_str => list of paths)."""
    by_day = defaultdict(list)
    for fname in sorted(os.listdir(month_nc_dir)):
        if fname.endswith(".nc") and "cyg" in fname:
            d = extract_date_from_filename(fname)
            if d:
                by_day[d].append(os.path.join(month_nc_dir, fname))
    return by_day

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def pick_existing_nc_month_dir(input_root, year, month):
    """
    Try multiple naming patterns for the NC month dir, both directly under root
    and under a year subfolder:
      - <root>/<YYYY_MM>, <root>/<YYYYMM>, <root>/<YYYY-MM>
      - <root>/<YYYY>/<YYYY_MM>, <root>/<YYYY>/<YYYYMM>, <root>/<YYYY>/<YYYY-MM>, <root>/<YYYY>/<MM>
    Return the first that exists, else None.
    """
    y = f"{year:04d}"
    ym_variants = [f"{year:04d}_{month:02d}", f"{year:04d}{month:02d}", f"{year:04d}-{month:02d}"]
    m_only = f"{month:02d}"

    candidates = []
    candidates += [os.path.join(input_root, v) for v in ym_variants]
    candidates += [os.path.join(input_root, y, v) for v in ym_variants]
    candidates.append(os.path.join(input_root, y, m_only))

    for p in candidates:
        if os.path.isdir(p):
            return p
    return None

def normalize_lon_in_df(df):
    """Ensure CSV longitudes are in [-180, 180] to match NC normalization."""
    if 'sp_lon' in df.columns:
        lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
        df['sp_lon'] = np.where(lon > 180, lon - 360, lon)

# ========= Build NC maps (union all NCs, exact keys + optional fallback) =========
def build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND):
    """
    Build from the UNION of all NCs for the day:
      - exact_map: (sample, ddm, rounded_lon, rounded_lat) -> tuple(values for NEW_VARS)
      - by_sd:     (sample, ddm) -> list of (raw_lon, raw_lat, *values_for_NEW_VARS) [for nearest fallback]
    Uses round(x, LON_LAT_ROUND) exactly like your demo code.
    Missing variables in a file are treated as NaN for that pixel.
    """
    exact_map = {}
    by_sd = defaultdict(list)
    dups_exact = 0

    # Track which requested vars exist anywhere this day (for info)
    seen_vars = set()

    for nc in nc_paths:
        try:
            ds = open_nc(nc)
        except Exception as e:
            print(f"  [WARN] Failed to open {nc}: {e}")
            continue

        try:
            samples = ds['sample'].values
            ddms    = ds['ddm'].values
            sp_lon  = norm_lon(ds['sp_lon'].values)
            sp_lat  = ds['sp_lat'].values

            if not (sp_lon.shape == sp_lat.shape):
                print("  [WARN] Shape mismatch for lon/lat; skipping this file.")
                continue

            Ni, Nj = sp_lon.shape
            ddms_int = ddms.astype(np.int64, copy=False)

            # Grab arrays for each NEW_VAR when present and matching shape
            var_arrays = {}
            for v in NEW_VARS:
                if v in ds.variables:
                    arr = ds[v].values
                    if getattr(arr, "shape", None) == (Ni, Nj):
                        var_arrays[v] = arr
                        seen_vars.add(v)

            # Iterate pixels and build maps
            for i in range(Ni):
                s_i = int(samples[i])
                for j in range(Nj):
                    d_j  = int(ddms_int[j])
                    lonr = round(float(sp_lon[i, j]), lonlat_round)
                    latr = round(float(sp_lat[i, j]), lonlat_round)
                    key  = (s_i, d_j, lonr, latr)

                    # Fixed-order values for NEW_VARS (NaN when missing)
                    vals = tuple(
                        (var_arrays[v][i, j] if v in var_arrays else np.nan)
                        for v in NEW_VARS
                    )

                    if key not in exact_map:
                        exact_map[key] = vals
                    else:
                        dups_exact += 1

                    by_sd[(s_i, d_j)].append(
                        (float(sp_lon[i, j]), float(sp_lat[i, j]), *vals)
                    )

        except KeyError as e:
            print(f"  [WARN] Missing required var in {nc}: {e}")
        finally:
            try:
                ds.close()
            except Exception:
                pass

    if dups_exact:
        print(f"  [INFO] {dups_exact:,} duplicate exact-key hits in the NC union; kept first.")

    missing_across_day = [v for v in NEW_VARS if v not in seen_vars]
    if missing_across_day:
        print(f"  [INFO] Not found in any NC for this day (filled as NaN): {missing_across_day}")

    return exact_map, by_sd

def nearest_by_lonlat(candidates, lon, lat):
    """
    candidates: list of (lon, lat, *values_for_NEW_VARS)
    Returns a tuple(values_for_NEW_VARS) for the candidate with min squared distance.
    """
    if not candidates:
        return tuple(np.nan for _ in NEW_VARS)
    # Compute distance on lon/lat only
    lons = np.fromiter((c[0] for c in candidates), dtype=float)
    lats = np.fromiter((c[1] for c in candidates), dtype=float)
    idx = int(np.argmin((lons - lon)**2 + (lats - lat)**2))
    chosen = candidates[idx]
    # chosen = (lon, lat, v0, v1, ..., v9)
    return tuple(chosen[2:2+len(NEW_VARS)])

# ========= CSV augmentation =========
def build_keys_from_df(df: pd.DataFrame):
    """Vectorized exact-key tuples from CSV columns, demo-style rounding."""
    s  = df['sample'].to_numpy(dtype=np.int64, copy=False)
    d  = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)
    lo = df['sp_lon'].to_numpy(dtype=float,   copy=False)
    la = df['sp_lat'].to_numpy(dtype=float,   copy=False)
    # demo-style: Python round() to LON_LAT_ROUND
    lo_r = np.array([round(x, LON_LAT_ROUND) for x in lo], dtype=float)
    la_r = np.array([round(x, LON_LAT_ROUND) for x in la], dtype=float)
    return list(zip(s, d, lo_r, la_r))

def augment_csv_for_day(csv_path, nc_paths, write_path):
    # Read CSV (nullable ints ok; use plain int64 if you never have NA)
    df = pd.read_csv(
        csv_path,
        dtype={'sample': 'Int64', 'ddm': 'Int64'},
        low_memory=False
    )
    if df.empty:
        print(f"  [INFO] Empty CSV, skipping: {os.path.basename(csv_path)}")
        if not SAFE_OVERWRITE:
            ensure_dir(os.path.dirname(write_path))
            df.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
        return

    required_cols = {'sample','ddm','sp_lon','sp_lat'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Required columns missing in {os.path.basename(csv_path)}: {sorted(missing)}")

    # Normalize CSV longitudes to [-180, 180] for consistency (matches NC)
    normalize_lon_in_df(df)

    # Build exact keys + arrays for fallback
    keys_list   = build_keys_from_df(df)
    csv_lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
    csv_lat = df['sp_lat'].to_numpy(dtype=float, copy=False)
    csv_s   = df['sample'].to_numpy(dtype=np.int64, copy=False)
    csv_d   = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)

    # Build NC maps for ALL NC files of the day (union)
    exact_map, by_sd = build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND)

    # Prepare per-variable collectors
    out_vals = {v: [] for v in NEW_VARS}
    miss_exact = miss_sd = 0

    for idx, k in enumerate(keys_list):
        vals = exact_map.get(k, None)
        if vals is None:
            miss_exact += 1  # exact miss
            cand = by_sd.get((int(csv_s[idx]), int(csv_d[idx])), [])
            vals = nearest_by_lonlat(cand, float(csv_lon[idx]), float(csv_lat[idx]))
            if all(isinstance(x, float) and np.isnan(x) for x in vals):
                miss_sd += 1  # still missing after nearest
        # Append into each column list by fixed order
        for vi, vname in enumerate(NEW_VARS):
            out_vals[vname].append(vals[vi])

    print(f"  [INFO] misses — exact:{miss_exact:,}  sd-nearest (still NaN tuple):{miss_sd:,}")

    # Assign columns (float32)
    for vname in NEW_VARS:
        df[vname] = pd.to_numeric(out_vals[vname], errors='coerce').astype('float32')

    # Write
    ensure_dir(os.path.dirname(write_path))
    compression = 'gzip' if USE_GZIP_OUTPUT else None
    df.to_csv(write_path, index=False, compression=compression)
    print(f"  [OK] Saved: {write_path}  (rows: {len(df):,})")

# ========= Main =========
def main():
    month_csv_dirs = find_month_dirs(output_root, years_filter)
    if not month_csv_dirs:
        print("[INFO] No month folders found under output_root that match your filter.")
        return

    for name, y, m, csv_month_dir in month_csv_dirs:
        print(f"\n=== Month {name} ===")

        # Find the matching NC dir for this month under input_root
        nc_month_dir = pick_existing_nc_month_dir(input_root, y, m)
        if not nc_month_dir:
            print(f"  [WARN] No NC month dir found under {input_root} for {y}-{m:02d}  (tried YYYY_MM, YYYYMM, YYYY-MM, and year subfolders)")
            continue

        day_to_nc = collect_nc_files_for_month(nc_month_dir)
        if not day_to_nc:
            print("  [INFO] No NCs with recognizable dates here; skipping month.")
            continue

        # Output dir
        if SAFE_OVERWRITE:
            out_month_dir = csv_month_dir
        else:
            out_month_dir = os.path.join(os.path.dirname(csv_month_dir), f"{y:04d}_{m:02d}{OUTPUT_SUFFIX}")

        # Iterate daily CSVs
        csv_files = [f for f in sorted(os.listdir(csv_month_dir)) if f.endswith(".csv") or f.endswith(".csv.gz")]
        if not csv_files:
            print("  [INFO] No CSVs in this month; skipping.")
            continue

        for fname in tqdm(csv_files, desc=f"{name}: augmenting"):
            mdate = re.match(r'^(\d{8})\.csv(\.gz)?$', fname)
            if not mdate:
                continue
            date_str = mdate.group(1)

            nc_paths = day_to_nc.get(date_str, [])
            src_csv  = os.path.join(csv_month_dir, fname)

            if SAFE_OVERWRITE:
                write_path = src_csv
            else:
                base_out = os.path.join(out_month_dir, f"{date_str}.csv")
                write_path = base_out + (".gz" if USE_GZIP_OUTPUT else "")

            if not nc_paths:
                print(f"  [WARN] No NCs found for {date_str}; leaving CSV unchanged.")
                if not SAFE_OVERWRITE:
                    ensure_dir(out_month_dir)
                    df_tmp = pd.read_csv(src_csv, low_memory=False)
                    df_tmp.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
                continue

            augment_csv_for_day(src_csv, nc_paths, write_path)

    print("\n[DONE] CSV augmentation complete.")

if __name__ == "__main__":
    main()



=== Month 2018_09 ===


2018_09: augmenting:   0%|                                                                                                              | 0/30 [00:00<?, ?it/s]

  [INFO] misses — exact:0  sd-nearest (still NaN tuple):0


2018_09: augmenting:   3%|███▍                                                                                                  | 1/30 [01:03<30:39, 63.42s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2018_09_v3/20180901.csv  (rows: 549,021)
  [INFO] misses — exact:0  sd-nearest (still NaN tuple):0


...

  [INFO] misses — exact:0  sd-nearest (still NaN tuple):0


2019_12: augmenting:  97%|██████████████████████████████████████████████████████████████████████████████▍  | 30/31 [1:15:47<02:35, 155.54s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_12_v3/20191230.csv  (rows: 1,496,454)
  [INFO] misses — exact:0  sd-nearest (still NaN tuple):0


2019_12: augmenting: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [1:18:27<00:00, 151.87s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_12_v3/20191231.csv  (rows: 1,505,235)

[DONE] CSV augmentation complete.


In [11]:
import pandas as pd
from pathlib import Path

ROUND_DECIMALS = 4
PREVIEW = False
NEEDED_COLS = ["modis_land_cover", "srtm_slope"]

def pick_join_keys(df_a: pd.DataFrame, df_b: pd.DataFrame):
    k1 = ["sample", "ddm"]
    if all(k in df_a.columns for k in k1) and all(k in df_b.columns for k in k1):
        return k1, "sample+ddm"
    k2 = ["sp_lon", "sp_lat"]
    if all(k in df_a.columns for k in k2) and all(k in df_b.columns for k in k2):
        return k2, "lon+lat"
    return None, "index"

def safe_read_csv(p: Path):
    try:
        return pd.read_csv(p)
    except Exception as e:
        print(f"[ERROR] reading {p}: {e}")
        return None

def ensure_outdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# --------- loop over 2020 months ----------
for month in range(1, 13):
    mm = f"{month:02d}"
    year_month = f"2023_{mm}"
    print(f"\n=== Processing {year_month} ===")

    CYGNSS_DIR = Path(f"/mnt/cephfs-mount/chenchen/CygnssDataCsvLand/{year_month}_v2")
    WATER_DIR  = Path(f"/mnt/cephfs-mount/chenchen/water_body_fraction/{year_month}")
    OUT_DIR    = Path(f"/mnt/cephfs-mount/chenchen/water_body_fraction_with_landcover&slope/{year_month}")

    ensure_outdir(OUT_DIR)

    water_files = sorted(WATER_DIR.glob("*.csv"))
    if not water_files:
        print(f"[ABORT] No CSVs found in {WATER_DIR}")
        continue

    for wpath in water_files:
        fname = wpath.name
        cpath = CYGNSS_DIR / fname

        wdf = safe_read_csv(wpath)
        if wdf is None:
            print(f"[SKIP] water read error: {fname}")
            continue

        if not cpath.exists():
            print(f"[SKIP] Missing CYGNSS: {fname}")
            continue

        cdf = safe_read_csv(cpath)
        if cdf is None:
            print(f"[SKIP] cygnss read error: {fname}")
            continue

        # Check CYGNSS has needed cols
        if any(col not in cdf.columns for col in NEEDED_COLS):
            print(f"[SKIP] {fname}: CYGNSS missing required cols")
            continue

        join_keys, key_name = pick_join_keys(wdf, cdf)

        if join_keys is None:
            # fallback index alignment
            min_len = min(len(wdf), len(cdf))
            if not PREVIEW:
                for col in NEEDED_COLS:
                    if col not in wdf.columns:
                        wdf[col] = pd.NA
                    wdf.loc[:min_len-1, col] = cdf.loc[:min_len-1, col].values
            matched, unmatched = min_len, len(wdf) - min_len
        else:
            wdf_keyed = wdf.copy()
            cdf_keyed = cdf.copy()
            if key_name == "lon+lat":
                for col in ["sp_lon", "sp_lat"]:
                    wdf_keyed[col] = wdf_keyed[col].round(ROUND_DECIMALS)
                    cdf_keyed[col] = cdf_keyed[col].round(ROUND_DECIMALS)
            cdf_keep = cdf_keyed[join_keys + NEEDED_COLS].drop_duplicates(subset=join_keys, keep="first")
            merged = wdf_keyed.merge(cdf_keep, on=join_keys, how="left", validate="m:1")
            matched = int(merged[NEEDED_COLS[0]].notna().sum())
            unmatched = len(merged) - matched
            if not PREVIEW:
                wdf = merged

        status = "preview" if PREVIEW else "ok"
        if not PREVIEW:
            out_path = OUT_DIR / fname
            try:
                wdf.to_csv(out_path, index=False)
            except Exception as e:
                print(f"[ERROR] writing {out_path}: {e}")
                status = "write_error"

        print(f"[{status}] {fname} | key={key_name} | matched {matched}/{len(wdf)}")

print("\n✅ All months (01–12) processed.")



=== Processing 2023_01 ===
[ok] 20230101.csv | key=sample+ddm | matched 1414/1414
[ok] 20230102.csv | key=sample+ddm | matched 1570/1570
[ok] 20230103.csv | key=sample+ddm | matched 1362/1362
[ok] 20230104.csv | key=sample+ddm | matched 1568/1568
[ok] 20230105.csv | key=sample+ddm | matched 1121/1121
[ok] 20230106.csv | key=sample+ddm | matched 1396/1396
[ok] 20230107.csv | key=sample+ddm | matched 1472/1472
[ok] 20230108.csv | key=sample+ddm | matched 1448/1448
[ok] 20230109.csv | key=sample+ddm | matched 1585/1585
[ok] 20230110.csv | key=sample+ddm | matched 1231/1231
[ok] 20230111.csv | key=sample+ddm | matched 1529/1529
[ok] 20230112.csv | key=sample+ddm | matched 1320/1320
[ok] 20230113.csv | key=sample+ddm | matched 1394/1394
[ok] 20230114.csv | key=sample+ddm | matched 1570/1570
[ok] 20230115.csv | key=sample+ddm | matched 1324/1324
[ok] 20230116.csv | key=sample+ddm | matched 1477/1477
[ok] 20230117.csv | key=sample+ddm | matched 1225/1225
[ok] 20230118.csv | key=sample+ddm | 

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, re, sys
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import xarray as xr

# ========= User settings =========
input_root   = "/mnt/cephfs-mount/chenchen/CygnssData"        # NC root (month folders)
output_root  = ("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full")  # Existing CSV root (month folders)
years_filter = {2020}  # e.g., {2023,2024} or None for all

# Write mode: overwrite or write to sibling folder
SAFE_OVERWRITE   = False
OUTPUT_SUFFIX    = "_v3"       # used if SAFE_OVERWRITE=False (writes YYYY_MM_v2)
USE_GZIP_OUTPUT  = False       # set True to write *.csv.gz

# Rounding for stable float keys (must match producer tolerance)
LON_LAT_ROUND = 6   # <- matches your demo

# ======== Pandas nullable-int detection / dtype helper ========
def get_nullable_int16_dtype():
    try:
        return pd.Int16Dtype()  # extension dtype (handles NaN)
    except Exception:
        return None

NULLABLE_INT16_DTYPE = get_nullable_int16_dtype()

# ========= Helpers =========
def parse_month_folder_name(name):
    m = re.match(r'^(\d{4})[-_]?(\d{2})$', name)
    if not m: return None, None
    y, mm = int(m.group(1)), int(m.group(2))
    return (y, mm) if 1 <= mm <= 12 else (None, None)

def find_month_dirs(root, years_filter=None):
    """Return (name, year, month, full_path). Accepts YYYYMM / YYYY_MM / YYYY-MM directly under root."""
    out = []
    for name in sorted(os.listdir(root)):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        y, m = parse_month_folder_name(name)
        if y is None:
            continue
        if (years_filter is None) or (y in years_filter):
            out.append((name, y, m, p))
    return out

def extract_date_from_filename(fname: str):
    m = re.search(r's(\d{8})', fname)
    return m.group(1) if m else None

def norm_lon(arr):
    # match your demo (> 180)
    return np.where(arr > 180, arr - 360, arr)

def open_nc(path):
    # Prefer netcdf4 engine if available
    try:
        return xr.open_dataset(path, engine='netcdf4', mask_and_scale=True)
    except Exception:
        return xr.open_dataset(path, mask_and_scale=True)

def collect_nc_files_for_month(month_nc_dir):
    """Find daily NC files (grouped by date_str => list of paths)."""
    by_day = defaultdict(list)
    for fname in sorted(os.listdir(month_nc_dir)):
        if fname.endswith(".nc") and "cyg" in fname:
            d = extract_date_from_filename(fname)
            if d:
                by_day[d].append(os.path.join(month_nc_dir, fname))
    return by_day

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def pick_existing_nc_month_dir(input_root, year, month):
    """
    Try multiple naming patterns for the NC month dir, both directly under root
    and under a year subfolder:
      - <root>/<YYYY_MM>, <root>/<YYYYMM>, <root>/<YYYY-MM>
      - <root>/<YYYY>/<YYYY_MM>, <root>/<YYYY>/<YYYYMM>, <root>/<YYYY>/<YYYY-MM>, <root>/<YYYY>/<MM>
    Return the first that exists, else None.
    """
    y = f"{year:04d}"
    ym_variants = [f"{year:04d}_{month:02d}", f"{year:04d}{month:02d}", f"{year:04d}-{month:02d}"]
    m_only = f"{month:02d}"

    candidates = []
    candidates += [os.path.join(input_root, v) for v in ym_variants]
    candidates += [os.path.join(input_root, y, v) for v in ym_variants]
    candidates.append(os.path.join(input_root, y, m_only))

    for p in candidates:
        if os.path.isdir(p):
            return p
    return None

def normalize_lon_in_df(df):
    """Ensure CSV longitudes are in [-180, 180] to match NC normalization."""
    if 'sp_lon' in df.columns:
        lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
        df['sp_lon'] = np.where(lon > 180, lon - 360, lon)

# ========= Build NC maps (union all NCs, demo-style exact keys + optional fallback) =========
def build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND):
    """
    Build from the UNION of all NCs for the day:
      - exact_map: (sample, ddm, rounded_lon, rounded_lat) -> (modis_land_cover, srtm_slope)
      - by_sd:     (sample, ddm) -> list of (raw_lon, raw_lat, lc, slope) [for nearest fallback]
    Uses round(x, LON_LAT_ROUND) exactly like your demo code.
    """
    exact_map = {}
    by_sd = defaultdict(list)
    missing_vars_warned = False
    dups_exact = 0
    total_pairs = 0

    for nc in nc_paths:
        try:
            ds = open_nc(nc)
        except Exception as e:
            print(f"  [WARN] Failed to open {nc}: {e}")
            continue

        try:
            samples = ds['sample'].values
            ddms    = ds['ddm'].values
            sp_lon  = norm_lon(ds['sp_lon'].values)
            sp_lat  = ds['sp_lat'].values

            if ('modis_land_cover' not in ds.variables) or ('srtm_slope' not in ds.variables):
                if not missing_vars_warned:
                    print("  [WARN] modis_land_cover and/or srtm_slope not found in an NC; skipping those files.")
                    missing_vars_warned = True
                continue

            lc    = ds['modis_land_cover'].values
            slope = ds['srtm_slope'].values

            if not (sp_lon.shape == sp_lat.shape == lc.shape == slope.shape):
                print("  [WARN] Shape mismatch among variables; skipping this file.")
                continue

            Ni, Nj = sp_lon.shape
            ddms_int = ddms.astype(np.int64, copy=False)
            total_pairs += Ni * Nj

            # demo-style: use Python round() to LON_LAT_ROUND
            for i in range(Ni):
                s_i = int(samples[i])
                for j in range(Nj):
                    d_j  = int(ddms_int[j])
                    lonr = round(float(sp_lon[i, j]), lonlat_round)
                    latr = round(float(sp_lat[i, j]), lonlat_round)
                    key  = (s_i, d_j, lonr, latr)

                    if key not in exact_map:
                        exact_map[key] = (lc[i, j], slope[i, j])
                    else:
                        dups_exact += 1

                    by_sd[(s_i, d_j)].append((float(sp_lon[i, j]), float(sp_lat[i, j]), lc[i, j], slope[i, j]))

        except KeyError as e:
            print(f"  [WARN] Missing required var in {nc}: {e}")
        finally:
            try:
                ds.close()
            except Exception:
                pass

    if dups_exact:
        print(f"  [INFO] {dups_exact:,} duplicate exact-key hits in the NC union; kept first.")
    # Optional: print union size (handy for debugging)
    # print(f"  [DEBUG] NC union exact keys: {len(exact_map):,}  (raw pairs: {total_pairs:,})")
    return exact_map, by_sd

def nearest_by_lonlat(candidates, lon, lat):
    """
    candidates: list of (lon, lat, lc, slope)
    Returns (lc, slope) for candidate with min squared distance in degrees.
    """
    if not candidates:
        return (np.nan, np.nan)
    arr = np.asarray(candidates, dtype=float)
    diffs = (arr[:,0] - lon)**2 + (arr[:,1] - lat)**2
    idx = int(np.argmin(diffs))
    return (arr[idx, 2], arr[idx, 3])

# ========= CSV augmentation =========
def build_keys_from_df(df: pd.DataFrame):
    """Vectorized exact-key tuples from CSV columns, demo-style rounding."""
    s  = df['sample'].to_numpy(dtype=np.int64, copy=False)
    d  = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)
    lo = df['sp_lon'].to_numpy(dtype=float,   copy=False)
    la = df['sp_lat'].to_numpy(dtype=float,   copy=False)
    # demo-style: Python round() to LON_LAT_ROUND
    lo_r = np.array([round(x, LON_LAT_ROUND) for x in lo], dtype=float)
    la_r = np.array([round(x, LON_LAT_ROUND) for x in la], dtype=float)
    return list(zip(s, d, lo_r, la_r))

def augment_csv_for_day(csv_path, nc_paths, write_path):
    # Read CSV (nullable ints ok; use plain int64 if you never have NA)
    df = pd.read_csv(
        csv_path,
        dtype={'sample': 'Int64', 'ddm': 'Int64'},
        low_memory=False
    )
    if df.empty:
        print(f"  [INFO] Empty CSV, skipping: {os.path.basename(csv_path)}")
        if not SAFE_OVERWRITE:
            ensure_dir(os.path.dirname(write_path))
            df.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
        return

    required_cols = {'sample','ddm','sp_lon','sp_lat'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Required columns missing in {os.path.basename(csv_path)}: {sorted(missing)}")

    # Normalize CSV longitudes to [-180, 180] for consistency (matches NC)
    normalize_lon_in_df(df)

    # Build exact keys + arrays for fallback
    keys_list   = build_keys_from_df(df)
    csv_lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
    csv_lat = df['sp_lat'].to_numpy(dtype=float, copy=False)
    csv_s   = df['sample'].to_numpy(dtype=np.int64, copy=False)
    csv_d   = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)

    # Build NC maps for ALL NC files of the day (union)
    exact_map, by_sd = build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND)

    # Map to new columns with cascade: exact -> (sample,ddm) nearest
    modis_vals = []
    slope_vals = []
    miss_exact = miss_sd = 0

    for idx, k in enumerate(keys_list):
        v = exact_map.get(k, None)
        if v is not None:
            modis_vals.append(v[0]); slope_vals.append(v[1])
            continue
        miss_exact += 1  # usually small/zero with the union build

        cand = by_sd.get((int(csv_s[idx]), int(csv_d[idx])), [])
        v2 = nearest_by_lonlat(cand, float(csv_lon[idx]), float(csv_lat[idx]))
        modis_vals.append(v2[0]); slope_vals.append(v2[1])
        if isinstance(v2[0], float) and np.isnan(v2[0]) and isinstance(v2[1], float) and np.isnan(v2[1]):
            miss_sd += 1

    print(f"  [INFO] misses — exact:{miss_exact:,}  sd-nearest (still NaN):{miss_sd:,}")

    # Downcast (with compatibility fallback)
    if NULLABLE_INT16_DTYPE is not None:
        df['modis_land_cover'] = pd.array(modis_vals, dtype=NULLABLE_INT16_DTYPE)
    else:
        df['modis_land_cover'] = pd.to_numeric(modis_vals, errors='coerce').astype('float32')
    df['srtm_slope'] = pd.to_numeric(slope_vals, errors='coerce').astype('float32')

    # Write
    ensure_dir(os.path.dirname(write_path))
    compression = 'gzip' if USE_GZIP_OUTPUT else None
    df.to_csv(write_path, index=False, compression=compression)
    print(f"  [OK] Saved: {write_path}  (rows: {len(df):,})")

# ========= Main =========
def main():
    month_csv_dirs = find_month_dirs(output_root, years_filter)
    if not month_csv_dirs:
        print("[INFO] No month folders found under output_root that match your filter.")
        return

    for name, y, m, csv_month_dir in month_csv_dirs:
        print(f"\n=== Month {name} ===")

        # Find the matching NC dir for this month under input_root
        nc_month_dir = pick_existing_nc_month_dir(input_root, y, m)
        if not nc_month_dir:
            print(f"  [WARN] No NC month dir found under {input_root} for {y}-{m:02d}  (tried YYYY_MM, YYYYMM, YYYY-MM, and year subfolders)")
            continue

        day_to_nc = collect_nc_files_for_month(nc_month_dir)
        if not day_to_nc:
            print("  [INFO] No NCs with recognizable dates here; skipping month.")
            continue

        # Output dir
        if SAFE_OVERWRITE:
            out_month_dir = csv_month_dir
        else:
            out_month_dir = os.path.join(os.path.dirname(csv_month_dir), f"{y:04d}_{m:02d}{OUTPUT_SUFFIX}")

        # Iterate daily CSVs
        csv_files = [f for f in sorted(os.listdir(csv_month_dir)) if f.endswith(".csv") or f.endswith(".csv.gz")]
        if not csv_files:
            print("  [INFO] No CSVs in this month; skipping.")
            continue

        for fname in tqdm(csv_files, desc=f"{name}: augmenting"):
            mdate = re.match(r'^(\d{8})\.csv(\.gz)?$', fname)
            if not mdate:
                continue
            date_str = mdate.group(1)

            nc_paths = day_to_nc.get(date_str, [])
            src_csv  = os.path.join(csv_month_dir, fname)

            if SAFE_OVERWRITE:
                write_path = src_csv
            else:
                base_out = os.path.join(out_month_dir, f"{date_str}.csv")
                write_path = base_out + (".gz" if USE_GZIP_OUTPUT else "")

            if not nc_paths:
                print(f"  [WARN] No NCs found for {date_str}; leaving CSV unchanged.")
                if not SAFE_OVERWRITE:
                    ensure_dir(out_month_dir)
                    df_tmp = pd.read_csv(src_csv, low_memory=False)
                    df_tmp.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
                continue

            augment_csv_for_day(src_csv, nc_paths, write_path)

    print("\n[DONE] CSV augmentation complete.")

if __name__ == "__main__":
    main()


=== Month 2020_01 ===


2020_01: augmenting:   0%|                                                                                             | 0/31 [00:00<?, ?it/s]

  [INFO] misses — exact:0  sd-nearest (still NaN):0


2020_01: augmenting:   3%|██▋                                                                               | 1/31 [02:05<1:02:38, 125.27s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2020_01_v3/20200101.csv  (rows: 1,496,623)
  [INFO] misses — exact:0  sd-nearest (still NaN):0


...

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2020_12_v3/20201229.csv  (rows: 1,315,355)
  [INFO] misses — exact:0  sd-nearest (still NaN):0


2020_12: augmenting:  97%|██████████████████████████████████████████████████████████████████████████████▍  | 30/31 [1:05:09<02:01, 121.25s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2020_12_v3/20201230.csv  (rows: 1,348,754)
  [INFO] misses — exact:0  sd-nearest (still NaN):0


2020_12: augmenting: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [1:07:11<00:00, 130.04s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2020_12_v3/20201231.csv  (rows: 1,338,099)

[DONE] CSV augmentation complete.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, re, sys
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import xarray as xr

# ========= User settings =========
input_root   = "/mnt/cephfs-mount/chenchen/CygnssData"        # NC root (month folders)
output_root  = ("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full")  # Existing CSV root (month folders)
years_filter = {2021}  # e.g., {2023,2024} or None for all

# Write mode: overwrite or write to sibling folder
SAFE_OVERWRITE   = False
OUTPUT_SUFFIX    = "_v3"       # used if SAFE_OVERWRITE=False (writes YYYY_MM_v2)
USE_GZIP_OUTPUT  = False       # set True to write *.csv.gz

# Rounding for stable float keys (must match producer tolerance)
LON_LAT_ROUND = 6   # <- matches your demo

# ======== Pandas nullable-int detection / dtype helper ========
def get_nullable_int16_dtype():
    try:
        return pd.Int16Dtype()  # extension dtype (handles NaN)
    except Exception:
        return None

NULLABLE_INT16_DTYPE = get_nullable_int16_dtype()

# ========= Helpers =========
def parse_month_folder_name(name):
    m = re.match(r'^(\d{4})[-_]?(\d{2})$', name)
    if not m: return None, None
    y, mm = int(m.group(1)), int(m.group(2))
    return (y, mm) if 1 <= mm <= 12 else (None, None)

def find_month_dirs(root, years_filter=None):
    """Return (name, year, month, full_path). Accepts YYYYMM / YYYY_MM / YYYY-MM directly under root."""
    out = []
    for name in sorted(os.listdir(root)):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        y, m = parse_month_folder_name(name)
        if y is None:
            continue
        if (years_filter is None) or (y in years_filter):
            out.append((name, y, m, p))
    return out

def extract_date_from_filename(fname: str):
    m = re.search(r's(\d{8})', fname)
    return m.group(1) if m else None

def norm_lon(arr):
    # match your demo (> 180)
    return np.where(arr > 180, arr - 360, arr)

def open_nc(path):
    # Prefer netcdf4 engine if available
    try:
        return xr.open_dataset(path, engine='netcdf4', mask_and_scale=True)
    except Exception:
        return xr.open_dataset(path, mask_and_scale=True)

def collect_nc_files_for_month(month_nc_dir):
    """Find daily NC files (grouped by date_str => list of paths)."""
    by_day = defaultdict(list)
    for fname in sorted(os.listdir(month_nc_dir)):
        if fname.endswith(".nc") and "cyg" in fname:
            d = extract_date_from_filename(fname)
            if d:
                by_day[d].append(os.path.join(month_nc_dir, fname))
    return by_day

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def pick_existing_nc_month_dir(input_root, year, month):
    """
    Try multiple naming patterns for the NC month dir, both directly under root
    and under a year subfolder:
      - <root>/<YYYY_MM>, <root>/<YYYYMM>, <root>/<YYYY-MM>
      - <root>/<YYYY>/<YYYY_MM>, <root>/<YYYY>/<YYYYMM>, <root>/<YYYY>/<YYYY-MM>, <root>/<YYYY>/<MM>
    Return the first that exists, else None.
    """
    y = f"{year:04d}"
    ym_variants = [f"{year:04d}_{month:02d}", f"{year:04d}{month:02d}", f"{year:04d}-{month:02d}"]
    m_only = f"{month:02d}"

    candidates = []
    candidates += [os.path.join(input_root, v) for v in ym_variants]
    candidates += [os.path.join(input_root, y, v) for v in ym_variants]
    candidates.append(os.path.join(input_root, y, m_only))

    for p in candidates:
        if os.path.isdir(p):
            return p
    return None

def normalize_lon_in_df(df):
    """Ensure CSV longitudes are in [-180, 180] to match NC normalization."""
    if 'sp_lon' in df.columns:
        lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
        df['sp_lon'] = np.where(lon > 180, lon - 360, lon)

# ========= Build NC maps (union all NCs, demo-style exact keys + optional fallback) =========
def build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND):
    """
    Build from the UNION of all NCs for the day:
      - exact_map: (sample, ddm, rounded_lon, rounded_lat) -> (modis_land_cover, srtm_slope)
      - by_sd:     (sample, ddm) -> list of (raw_lon, raw_lat, lc, slope) [for nearest fallback]
    Uses round(x, LON_LAT_ROUND) exactly like your demo code.
    """
    exact_map = {}
    by_sd = defaultdict(list)
    missing_vars_warned = False
    dups_exact = 0
    total_pairs = 0

    for nc in nc_paths:
        try:
            ds = open_nc(nc)
        except Exception as e:
            print(f"  [WARN] Failed to open {nc}: {e}")
            continue

        try:
            samples = ds['sample'].values
            ddms    = ds['ddm'].values
            sp_lon  = norm_lon(ds['sp_lon'].values)
            sp_lat  = ds['sp_lat'].values

            if ('modis_land_cover' not in ds.variables) or ('srtm_slope' not in ds.variables):
                if not missing_vars_warned:
                    print("  [WARN] modis_land_cover and/or srtm_slope not found in an NC; skipping those files.")
                    missing_vars_warned = True
                continue

            lc    = ds['modis_land_cover'].values
            slope = ds['srtm_slope'].values

            if not (sp_lon.shape == sp_lat.shape == lc.shape == slope.shape):
                print("  [WARN] Shape mismatch among variables; skipping this file.")
                continue

            Ni, Nj = sp_lon.shape
            ddms_int = ddms.astype(np.int64, copy=False)
            total_pairs += Ni * Nj

            # demo-style: use Python round() to LON_LAT_ROUND
            for i in range(Ni):
                s_i = int(samples[i])
                for j in range(Nj):
                    d_j  = int(ddms_int[j])
                    lonr = round(float(sp_lon[i, j]), lonlat_round)
                    latr = round(float(sp_lat[i, j]), lonlat_round)
                    key  = (s_i, d_j, lonr, latr)

                    if key not in exact_map:
                        exact_map[key] = (lc[i, j], slope[i, j])
                    else:
                        dups_exact += 1

                    by_sd[(s_i, d_j)].append((float(sp_lon[i, j]), float(sp_lat[i, j]), lc[i, j], slope[i, j]))

        except KeyError as e:
            print(f"  [WARN] Missing required var in {nc}: {e}")
        finally:
            try:
                ds.close()
            except Exception:
                pass

    if dups_exact:
        print(f"  [INFO] {dups_exact:,} duplicate exact-key hits in the NC union; kept first.")
    # Optional: print union size (handy for debugging)
    # print(f"  [DEBUG] NC union exact keys: {len(exact_map):,}  (raw pairs: {total_pairs:,})")
    return exact_map, by_sd

def nearest_by_lonlat(candidates, lon, lat):
    """
    candidates: list of (lon, lat, lc, slope)
    Returns (lc, slope) for candidate with min squared distance in degrees.
    """
    if not candidates:
        return (np.nan, np.nan)
    arr = np.asarray(candidates, dtype=float)
    diffs = (arr[:,0] - lon)**2 + (arr[:,1] - lat)**2
    idx = int(np.argmin(diffs))
    return (arr[idx, 2], arr[idx, 3])

# ========= CSV augmentation =========
def build_keys_from_df(df: pd.DataFrame):
    """Vectorized exact-key tuples from CSV columns, demo-style rounding."""
    s  = df['sample'].to_numpy(dtype=np.int64, copy=False)
    d  = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)
    lo = df['sp_lon'].to_numpy(dtype=float,   copy=False)
    la = df['sp_lat'].to_numpy(dtype=float,   copy=False)
    # demo-style: Python round() to LON_LAT_ROUND
    lo_r = np.array([round(x, LON_LAT_ROUND) for x in lo], dtype=float)
    la_r = np.array([round(x, LON_LAT_ROUND) for x in la], dtype=float)
    return list(zip(s, d, lo_r, la_r))

def augment_csv_for_day(csv_path, nc_paths, write_path):
    # Read CSV (nullable ints ok; use plain int64 if you never have NA)
    df = pd.read_csv(
        csv_path,
        dtype={'sample': 'Int64', 'ddm': 'Int64'},
        low_memory=False
    )
    if df.empty:
        print(f"  [INFO] Empty CSV, skipping: {os.path.basename(csv_path)}")
        if not SAFE_OVERWRITE:
            ensure_dir(os.path.dirname(write_path))
            df.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
        return

    required_cols = {'sample','ddm','sp_lon','sp_lat'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Required columns missing in {os.path.basename(csv_path)}: {sorted(missing)}")

    # Normalize CSV longitudes to [-180, 180] for consistency (matches NC)
    normalize_lon_in_df(df)

    # Build exact keys + arrays for fallback
    keys_list   = build_keys_from_df(df)
    csv_lon = df['sp_lon'].to_numpy(dtype=float, copy=False)
    csv_lat = df['sp_lat'].to_numpy(dtype=float, copy=False)
    csv_s   = df['sample'].to_numpy(dtype=np.int64, copy=False)
    csv_d   = df['ddm'   ].to_numpy(dtype=np.int64, copy=False)

    # Build NC maps for ALL NC files of the day (union)
    exact_map, by_sd = build_value_maps_for_day(nc_paths, lonlat_round=LON_LAT_ROUND)

    # Map to new columns with cascade: exact -> (sample,ddm) nearest
    modis_vals = []
    slope_vals = []
    miss_exact = miss_sd = 0

    for idx, k in enumerate(keys_list):
        v = exact_map.get(k, None)
        if v is not None:
            modis_vals.append(v[0]); slope_vals.append(v[1])
            continue
        miss_exact += 1  # usually small/zero with the union build

        cand = by_sd.get((int(csv_s[idx]), int(csv_d[idx])), [])
        v2 = nearest_by_lonlat(cand, float(csv_lon[idx]), float(csv_lat[idx]))
        modis_vals.append(v2[0]); slope_vals.append(v2[1])
        if isinstance(v2[0], float) and np.isnan(v2[0]) and isinstance(v2[1], float) and np.isnan(v2[1]):
            miss_sd += 1

    print(f"  [INFO] misses — exact:{miss_exact:,}  sd-nearest (still NaN):{miss_sd:,}")

    # Downcast (with compatibility fallback)
    if NULLABLE_INT16_DTYPE is not None:
        df['modis_land_cover'] = pd.array(modis_vals, dtype=NULLABLE_INT16_DTYPE)
    else:
        df['modis_land_cover'] = pd.to_numeric(modis_vals, errors='coerce').astype('float32')
    df['srtm_slope'] = pd.to_numeric(slope_vals, errors='coerce').astype('float32')

    # Write
    ensure_dir(os.path.dirname(write_path))
    compression = 'gzip' if USE_GZIP_OUTPUT else None
    df.to_csv(write_path, index=False, compression=compression)
    print(f"  [OK] Saved: {write_path}  (rows: {len(df):,})")

# ========= Main =========
def main():
    month_csv_dirs = find_month_dirs(output_root, years_filter)
    if not month_csv_dirs:
        print("[INFO] No month folders found under output_root that match your filter.")
        return

    for name, y, m, csv_month_dir in month_csv_dirs:
        print(f"\n=== Month {name} ===")

        # Find the matching NC dir for this month under input_root
        nc_month_dir = pick_existing_nc_month_dir(input_root, y, m)
        if not nc_month_dir:
            print(f"  [WARN] No NC month dir found under {input_root} for {y}-{m:02d}  (tried YYYY_MM, YYYYMM, YYYY-MM, and year subfolders)")
            continue

        day_to_nc = collect_nc_files_for_month(nc_month_dir)
        if not day_to_nc:
            print("  [INFO] No NCs with recognizable dates here; skipping month.")
            continue

        # Output dir
        if SAFE_OVERWRITE:
            out_month_dir = csv_month_dir
        else:
            out_month_dir = os.path.join(os.path.dirname(csv_month_dir), f"{y:04d}_{m:02d}{OUTPUT_SUFFIX}")

        # Iterate daily CSVs
        csv_files = [f for f in sorted(os.listdir(csv_month_dir)) if f.endswith(".csv") or f.endswith(".csv.gz")]
        if not csv_files:
            print("  [INFO] No CSVs in this month; skipping.")
            continue

        for fname in tqdm(csv_files, desc=f"{name}: augmenting"):
            mdate = re.match(r'^(\d{8})\.csv(\.gz)?$', fname)
            if not mdate:
                continue
            date_str = mdate.group(1)

            nc_paths = day_to_nc.get(date_str, [])
            src_csv  = os.path.join(csv_month_dir, fname)

            if SAFE_OVERWRITE:
                write_path = src_csv
            else:
                base_out = os.path.join(out_month_dir, f"{date_str}.csv")
                write_path = base_out + (".gz" if USE_GZIP_OUTPUT else "")

            if not nc_paths:
                print(f"  [WARN] No NCs found for {date_str}; leaving CSV unchanged.")
                if not SAFE_OVERWRITE:
                    ensure_dir(out_month_dir)
                    df_tmp = pd.read_csv(src_csv, low_memory=False)
                    df_tmp.to_csv(write_path, index=False, compression=('gzip' if USE_GZIP_OUTPUT else None))
                continue

            augment_csv_for_day(src_csv, nc_paths, write_path)

    print("\n[DONE] CSV augmentation complete.")

if __name__ == "__main__":
    main()


=== Month 2021_01 ===


2021_01: augmenting:   0%|                                                                                             | 0/31 [00:00<?, ?it/s]

  [INFO] misses — exact:0  sd-nearest (still NaN):0


2021_01: augmenting:   3%|██▋                                                                                 | 1/31 [01:57<58:40, 117.35s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2021_01_v3/20210101.csv  (rows: 1,300,698)
  [INFO] misses — exact:0  sd-nearest (still NaN):0


...

2021_05: augmenting:  97%|██████████████████████████████████████████████████████████████████████████████▍  | 30/31 [1:06:01<02:07, 127.97s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2021_05_v3/20210530.csv  (rows: 1,485,988)
  [INFO] misses — exact:0  sd-nearest (still NaN):0


2021_05: augmenting: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [1:08:05<00:00, 131.78s/it]

  [OK] Saved: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2021_05_v3/20210531.csv  (rows: 1,496,302)

=== Month 2021_06 ===



2021_06: augmenting:   0%|                                                                                             | 0/30 [00:00<?, ?it/s]

In [3]:
import os, re

CSV_MONTH_DIR = "/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2023_12_v2"
NC_MONTH_DIR  = "/mnt/cephfs-mount/chenchen/CygnssData/202312"

# CSV dates present in that v2 folder
csv_dates = set()
for f in os.listdir(CSV_MONTH_DIR):
    m = re.match(r"^(\d{8})\.csv(\.gz)?$", f)
    if m:
        csv_dates.add(m.group(1))

# NC dates present in that month
nc_dates = set()
for f in os.listdir(NC_MONTH_DIR):
    if f.endswith(".nc"):
        m = re.search(r"s(\d{8})", f)
        if m:
            nc_dates.add(m.group(1))

print("CSV-only dates (no NC found):", sorted(csv_dates - nc_dates))
print("NC-only dates (no CSV present):", sorted(nc_dates - csv_dates))
print("Common dates:", len(csv_dates & nc_dates))

CSV-only dates (no NC found): []
NC-only dates (no CSV present): []
Common dates: 31
